In [ ]:
import pandas as pd
import numpy as np
import json
import os
from datetime import datetime


df_existing = pd.read_parquet("/content/drive/MyDrive/Projet_Deep_learning/Api_yaho/eur_usd_max_data.parquet")


filename = '/content/drive/MyDrive/Projet_Deep_learning/notebook_model_classification/output_models.json'


data = {} 

if os.path.exists(filename):
    try:
        with open(filename, 'r') as json_file:
            content = json_file.read()
            if content.strip(): 
                try:
                    data = json.loads(content) 
                    print("Contenu JSON valide :", data)
                    if not isinstance(data, dict):
                        print("Les données JSON doivent être un dictionnaire.")
                        data = {} 
                except json.JSONDecodeError as e:
                    print(f"Erreur dans le format JSON : {e}")
            else:
                print("Le fichier est vide.")
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
else:
    print(f"Le fichier {filename} n'existe pas.")


if data:  
    open_price = data.get('open', 0.0)  
    print(f"le open est : {open_price}")
    high_price = data.get('High', 0.0)  
    print(f"le high est : {high_price}")  
    low_price = data.get('low', 0.0)  
    close_price = data.get('Close', 0.0) 
else:
    open_price = 0.0
    high_price = 0.0
    low_price = 0.0
    close_price = 0.0

new_record = {
    'Date': datetime.now().strftime('%Y-%m-%d'),
    'Open': open_price,
    'High': high_price,
    'Low': low_price,
    'Close': close_price
}

data = []  
data.append(new_record)

df_new = pd.DataFrame(data)

df_combined = pd.concat([df_existing, df_new], ignore_index=True)

if len(df_combined) >= 35: 
    df_combined['diff_open_close'] = df_combined['Open'] - df_combined['Close']
    df_combined['SMA_10'] = df_combined['Close'].rolling(window=10).mean()
    df_combined['SMA_50'] = df_combined['Close'].rolling(window=50).mean()
    df_combined['EMA_12'] = df_combined['Close'].ewm(span=12, adjust=False).mean()
    df_combined['EMA_26'] = df_combined['Close'].ewm(span=26, adjust=False).mean()
    df_combined['MACD'] = df_combined['EMA_12'] - df_combined['EMA_26']
    df_combined['Signal_Line'] = df_combined['MACD'].ewm(span=9, adjust=False).mean()

    # Fonction pour calculer le RSI
    def calculate_rsi(df, window=14):
        delta = df['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        return rsi

    df_combined['RSI'] = calculate_rsi(df_combined)
    df_combined['price_range'] = df_combined['High'] - df_combined['Low']

    def calculate_atr(df, window=14):
        high_low = df['High'] - df['Low']
        high_close = np.abs(df['High'] - df['Close'].shift())
        low_close = np.abs(df['Low'] - df['Close'].shift())
        ranges = pd.concat([high_low, high_close, low_close], axis=1)
        true_range = ranges.max(axis=1)
        atr = true_range.rolling(window=window, min_periods=1).mean()
        return atr

    df_combined['ATR'] = calculate_atr(df_combined)

    def calculate_stochastic_oscillator(df, window=10):
        low_min = df['Low'].rolling(window=window).min()
        high_max = df['High'].rolling(window=window).max()
        stochastic = (df['Close'] - low_min) / (high_max - low_min) * 100
        return stochastic

    df_combined['Stochastic_Oscillator'] = calculate_stochastic_oscillator(df_combined)

    latest_features = df_combined.iloc[-1][['diff_open_close', 'SMA_10', 'MACD', 'RSI', 'price_range', 'ATR', 'Stochastic_Oscillator']]
    print("Dernières caractéristiques du nouvel enregistrement:")
    print(latest_features)
else:
    print("Pas assez de données pour effectuer les calculs des caractéristiques.")


In [1]:
import joblib
import numpy as np
import pandas as pd

try:
    scaler_RF = joblib.load('/content/drive/MyDrive/Projet_Deep_learning/models/scaler_RF.pkl')
    model_RF = joblib.load('/content/drive/MyDrive/Projet_Deep_learning/models/model_RF.pkl')
    print("Modèle et scaler chargés avec succès.")
except Exception as e:
    print(f"Erreur lors du chargement : {str(e)}")  

if 'scaler_RF' in locals() and 'model_RF' in locals():

    new_data = pd.DataFrame([latest_features])
    new_data_scaled = scaler_RF.transform(new_data)
    y_pred_new = model_RF.predict(new_data_scaled)
    action = "Buy" if y_pred_new[0] == 1 else "Sell"
    print(f"Prediction for new data: {action}")
    
    result = {"action": action}
    json_file_path = '/content/drive/MyDrive/Projet_Deep_learning/notebook_model_classification/output_models.json'
    
    if os.path.exists(json_file_path):
        with open(json_file_path, 'r+') as json_file:
            data = json.load(json_file)
            data.update(result)  
            json_file.seek(0) 
            json.dump(data, json_file, indent=4)
            json_file.truncate()  
    else:
        with open(json_file_path, 'w') as json_file:
            json.dump(result, json_file, indent=4)
else:
    print("Les modèles n'ont pas été chargés correctement, vérifiez les fichiers.")


Erreur lors du chargement : name 'pickle' is not defined
Les modèles n'ont pas été chargés correctement, vérifiez les fichiers.
